<a href="https://colab.research.google.com/github/RealMyeong/Aiffel_Exploration/blob/main/EX_BS2_project13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

In [1]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.6
1.7.3
0.6.1


1) 데이터 준비와 전처리
---
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [3]:
import pandas as pd
rating_file_path='/content/drive/MyDrive/AIFFEL/EX/13.좋아하는 아티스트 찾기/project/data_movie/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 3점 이상만 남깁니다.
ratings_2 = ratings.copy()
ratings_2 = ratings_2[ratings_2['ratings']>=3]
filtered_data_size = len(ratings_2)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
ratings_2.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
ratings_2.rename(columns={'ratings':'counts'}, inplace=True)

In [7]:
ratings_2['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [8]:
ratings_2.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


timestamp는 필요 없을 것 같다.

In [9]:
use_columns = ['user_id', 'movie_id', 'counts']
ratings_2 = ratings_2[use_columns]
ratings_2.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [10]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path='/content/drive/MyDrive/AIFFEL/EX/13.좋아하는 아티스트 찾기/project/data_movie/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


영화 제목과 movie_id를 한 번에 보기 위해 데이터 합침

In [11]:
merge_data = pd.merge(ratings_2, movies, on='movie_id')

In [12]:
merge_data.tail()

,user_id,movie_id,counts,title,genre
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5,Identification of a Woman (Identificazione di ...,Drama


In [13]:
all_data = merge_data.copy()

In [14]:
all_data.head()

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama


In [15]:
all_data.tail()

,user_id,movie_id,counts,title,genre
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5,Identification of a Woman (Identificazione di ...,Drama


첫 번째 유저가 어떤 영화를 봤는지 확인

In [16]:
condition = (all_data['user_id']== all_data.loc[0, 'user_id'])
all_data.loc[condition]

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1680,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2123,1,914,3,My Fair Lady (1964),Musical|Romance
2734,1,3408,4,Erin Brockovich (2000),Drama
3957,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
5556,1,1197,3,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
7808,1,1287,5,Ben-Hur (1959),Action|Adventure|Drama
8474,1,2804,5,"Christmas Story, A (1983)",Comedy|Drama
9764,1,594,4,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
10471,1,919,4,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical


## 데이터 분석

데이터 탐색
---
추천 모델을 만들기 전에 데이터의 기본적인 정보를 보고 갑시다. 아래 항목들을 확인해 보고 싶습니다.

- 유저 수, 영화 수, 인기 많은 영화
- 유저들이 몇 개의 영화를 보고 있는지에 대한 통계  
(참고) pandas.DataFrame.nunique()은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용합니다.

In [17]:
# 유저 수
a = all_data['user_id'].nunique()
# 영화 개수
b = all_data['title'].nunique()

print(f'총 유저 수 : {a}')
print(f'총 영화 개수 : {b}')

총 유저 수 : 6039
총 영화 개수 : 3628


In [18]:
all_data['user_id'].max()

6040

In [19]:
# 가장 인기가 많은 영화
m_count = all_data.groupby('title')['user_id'].count()
m_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

가장 인기있는 영화 30개를 뽑아보면 실제로 보지는 못했어도 이름은 들어본 근본 영화들이 많다.

In [20]:
# 유저 별 영화를 몇개정도 보는지
user_count = all_data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

각 유저마다 영화를 얼마나 보는지에 대해 봤는데 1968개를 본 사람은...? 

## 선호하는 영화 5개 추가해보기

내가 마음에 드는 영화로 5개를 골라 기존의 데이터에 넣어준다.

In [21]:
# 영화 제목으로 장르 추출하는 함수
def get_genre_of_movie(my_favorite):
    genres_of_my_favorite = []
    for k, v in my_favorite.items():
        genres_of_my_favorite.extend(movies[movies['title'] == v]['genre'].to_list())
    return genres_of_my_favorite

In [22]:
# 영화 데이터 추가! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요.
my_favorite = {
    70: "From Dusk Till Dawn (1996)",
    164: "Devil in a Blue Dress (1995)",
    293: "Professional, The (a.k.a. Leon: The Professional) (1994)",
    426: "Body Snatchers (1993)",
    3476: "Jacob's Ladder (1990)",
}

# 6041이라는 user_id가 위 영화를 counts회씩 시청했다고 가정하겠습니다.
my_playlist = pd.DataFrame({
    'user_id': [6041]*5,
    'movie_id': my_favorite.keys(),
    'counts': [3,4,4,5,5],
    'title': my_favorite.values(),
    'genre': get_genre_of_movie(my_favorite)
})

if not all_data.isin({'user_id':[6041]})['user_id'].any():  # user_id에 6041이라는 데이터가 없다면
    all_data = all_data.append(my_playlist)  # my_favorite 데이터 추가

all_data.tail(10)  # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title,genre
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5,Identification of a Woman (Identificazione di ...,Drama
0,6041,70,3,From Dusk Till Dawn (1996),Action|Comedy|Crime|Horror|Thriller
1,6041,164,4,Devil in a Blue Dress (1995),Crime|Film-Noir|Mystery|Thriller
2,6041,293,4,"Professional, The (a.k.a. Leon: The Profession...",Crime|Drama|Romance|Thriller
3,6041,426,5,Body Snatchers (1993),Horror|Sci-Fi|Thriller
4,6041,3476,5,Jacob's Ladder (1990),Horror|Mystery|Thriller


- 한 영화에도 여러개의 genre가 섞여있는 영화가 있기 때문에 장르의 종류가 얼마나 되는지 리스트와 Counter를 이용해서 계산해줌

In [23]:
# from collections import Counter

# genre_list = []
# for i in all_data['genre']:
#   for j in range(len(i)):
#     genre_list.append(i[j])

# genre_count = Counter(genre_list)
# genre_count

In [24]:
# genre_count.keys()

총 장르의 종류는 18종류이다. 여러개의 장르가 섞여 있는 영화의 장르를 어떻게 표현 할까 생각하다가 처음에는 one-hot encoding으로 해볼까 라는 생각을 했지만 데이터가 너무 커질 것 같아서 애초에 영화를 만들 때 genre가 섞여서 만들어지기 때문에 각 장르마다 index를 부여

In [25]:
# genre_index = {'Drama': 1,
#          'Animation': 2,
#          "Children's": 3,
#          'Musical': 4,
#          'Romance': 5,
#          'Comedy': 6,
#          'Action': 7,
#          'Adventure': 8,
#          'Fantasy': 9,
#          'Sci-Fi': 10,
#          'War': 11,
#          'Thriller': 12,
#          'Crime': 13,
#          'Mystery': 14,
#          'Western': 15,
#          'Horror': 16,
#          'Film-Noir': 17,
#          'Documentary': 18}

# # 각 장르를 정해둔 인덱스값으로 바꿔줌
# for i in all_data['genre']:
#   for j in range(len(i)):
#     i[j] = genre_index[i[j]]
  

In [26]:
all_data.tail(20)

,user_id,movie_id,counts,title,genre
836463,5328,2438,4,Outside Ozona (1998),Drama|Thriller
836464,5334,3323,3,Chain of Fools (2000),Comedy|Crime
836465,5334,3382,5,Song of Freedom (1936),Drama
836466,5420,1843,3,Slappy and the Stinkers (1998),Children's|Comedy
836467,5433,286,3,Nemesis 2: Nebula (1995),Action|Sci-Fi|Thriller
836468,5494,3530,4,Smoking/No Smoking (1993),Comedy
836469,5556,2198,3,Modulations (1998),Documentary
836470,5949,2198,5,Modulations (1998),Documentary
836471,5675,2703,3,Broken Vessels (1998),Drama
836472,5717,2258,4,Master Ninja I (1984),Action


In [27]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836483 entries, 0 to 4
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   836483 non-null  int64 
 1   movie_id  836483 non-null  int64 
 2   counts    836483 non-null  int64 
 3   title     836483 non-null  object
 4   genre     836483 non-null  object
dtypes: int64(3), object(2)
memory usage: 38.3+ MB


각 장르가 하나의 숫자로 잘 바뀐것을 볼 수 있다.


모델에서 활용하기 위해 인덱싱 과정 

In [28]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = all_data['user_id'].unique()
movie_unique = all_data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [29]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print('user_id indexing: {}\n'.format(user_to_idx[6041]))

for k, v in my_favorite.items():
    print('movie_id indexing: {}'.format(movie_to_idx[v]))

user_id indexing: 6039

movie_id indexing: 1034
movie_id indexing: 864
movie_id indexing: 890
movie_id indexing: 2361
movie_id indexing: 279


In [30]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_all_data = all_data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_all_data) == len(all_data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    all_data['user_id'] = temp_user_all_data   # all_data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
# temp_movie_all_data = all_data['movie_id'].map(movie_to_idx.get).dropna()
temp_movie_all_data = all_data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_all_data) == len(all_data):
    print('movie column indexing OK!!')
    all_data['movie_id'] = temp_movie_all_data
else:
    print('movie column indexing Fail!!')

all_data.tail(10)

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts,title,genre
836473,1621,3623,5,One Little Indian (1973),Comedy|Drama|Western
836474,3481,3624,4,Slaughterhouse (1987),Horror
836475,3481,3625,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,4159,3626,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,1648,3627,5,Identification of a Woman (Identificazione di ...,Drama
0,6039,1034,3,From Dusk Till Dawn (1996),Action|Comedy|Crime|Horror|Thriller
1,6039,864,4,Devil in a Blue Dress (1995),Crime|Film-Noir|Mystery|Thriller
2,6039,890,4,"Professional, The (a.k.a. Leon: The Profession...",Crime|Drama|Romance|Thriller
3,6039,2361,5,Body Snatchers (1993),Horror|Sci-Fi|Thriller
4,6039,279,5,Jacob's Ladder (1990),Horror|Mystery|Thriller


In [31]:
all_data.tail(10)

,user_id,movie_id,counts,title,genre
836473,1621,3623,5,One Little Indian (1973),Comedy|Drama|Western
836474,3481,3624,4,Slaughterhouse (1987),Horror
836475,3481,3625,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,4159,3626,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,1648,3627,5,Identification of a Woman (Identificazione di ...,Drama
0,6039,1034,3,From Dusk Till Dawn (1996),Action|Comedy|Crime|Horror|Thriller
1,6039,864,4,Devil in a Blue Dress (1995),Crime|Film-Noir|Mystery|Thriller
2,6039,890,4,"Professional, The (a.k.a. Leon: The Profession...",Crime|Drama|Romance|Thriller
3,6039,2361,5,Body Snatchers (1993),Horror|Sci-Fi|Thriller
4,6039,279,5,Jacob's Ladder (1990),Horror|Mystery|Thriller


In [32]:
all_data['movie_id'].nunique(), all_data['genre'].nunique()

(3628, 301)

~~영화의 제목으로 추천을 해주는 것 보다는 영화의 장르에 따라서 추천을 해주는 게 좋을 것 같음.
따라서 movie_id는 사용하지않고 genre만 사용~~

genre로 사용하려고 했는데 데이터가 너무 줄어버림 -> 아마 같은 장르지만 제목만 다른 영화들이 있어서 그런 것 같다. 크게 의미 없는 행동 이었던 걸로... ㅎㅎ

In [33]:
all_data.drop(['genre'], axis=1, inplace=True)

In [34]:
all_data.tail(10)

,user_id,movie_id,counts,title
836473,1621,3623,5,One Little Indian (1973)
836474,3481,3624,4,Slaughterhouse (1987)
836475,3481,3625,3,"Promise, The (Versprechen, Das) (1994)"
836476,4159,3626,4,"Five Wives, Three Secretaries and Me (1998)"
836477,1648,3627,5,Identification of a Woman (Identificazione di ...
0,6039,1034,3,From Dusk Till Dawn (1996)
1,6039,864,4,Devil in a Blue Dress (1995)
2,6039,890,4,"Professional, The (a.k.a. Leon: The Profession..."
3,6039,2361,5,Body Snatchers (1993)
4,6039,279,5,Jacob's Ladder (1990)


In [35]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836483 entries, 0 to 4
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   836483 non-null  int64 
 1   movie_id  836483 non-null  int64 
 2   counts    836483 non-null  int64 
 3   title     836483 non-null  object
dtypes: int64(3), object(1)
memory usage: 31.9+ MB


In [36]:
all_data.isnull().sum()

user_id     0
movie_id    0
counts      0
title       0
dtype: int64

## CSR matrix 생성

In [37]:
from scipy.sparse import csr_matrix

num_user = all_data['user_id'].nunique()
num_movie = all_data['movie_id'].nunique()

csr_data = csr_matrix((all_data.counts, (all_data.user_id, all_data.movie_id)), shape = (num_user, num_movie))

In [38]:
csr_data.shape

(6040, 3628)

## MF 모델 학습하기

In [39]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보겠습니다.

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지  

1과 4를 늘릴수록 학습 데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [40]:
# implicit alternatingLeastSquares 모델 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [41]:
#als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [42]:
#모델 훈련
als_model.fit(csr_data)

  0%|          | 0/20 [00:00<?, ?it/s]

## 영화의 선호도 예측

In [43]:
user = user_to_idx[6041]
user_vector = als_model.user_factors[user]
user_vector

array([-3.58635157e-01, -2.30420768e-01, -2.23755330e-01,  5.07999897e-01,
        2.05090493e-01,  5.43885026e-03, -1.11529104e-01, -6.52697027e-01,
       -1.41748875e-01,  1.80312321e-01, -6.76423132e-01, -5.60070090e-02,
       -6.03624523e-01,  2.04060316e-01,  2.67539680e-01,  3.87154400e-01,
       -3.79003346e-01, -7.02822328e-01,  4.29998577e-01,  5.88361323e-01,
       -9.18122903e-02, -4.17757407e-02,  4.49920833e-01,  2.63408780e-01,
        6.81922585e-02,  3.84306312e-01,  4.22169417e-01,  6.08847737e-01,
        4.57840383e-01,  2.06231579e-01, -1.44827679e-01,  1.86764970e-01,
        1.58368930e-01,  5.85599065e-01,  4.89072919e-01,  3.42123300e-01,
        2.92201228e-02, -5.76993711e-02, -4.65202421e-01, -2.57920116e-01,
       -1.89551711e-01,  7.70712972e-01,  4.87679094e-01,  3.27985257e-01,
       -7.36922994e-02,  3.00199632e-02, -4.43425119e-01,  1.33759886e-01,
        1.51406124e-01, -6.47950351e-01,  6.02377772e-01,  8.68232325e-02,
       -4.17043716e-01, -

In [44]:
star_wars_epi_4 = movie_to_idx['Star Wars: Episode IV - A New Hope (1977)']
star_wars_epi_4_vector = als_model.item_factors[star_wars_epi_4]
star_wars_epi_4_vector  # 영화 벡터

array([ 3.69643942e-02, -1.33783184e-02,  1.30297402e-02, -2.18583904e-02,
        2.60371007e-02, -6.70848507e-03,  5.13088284e-03, -2.39761528e-02,
       -8.15320294e-03,  8.66318773e-03,  4.13981266e-02,  2.24568937e-02,
       -2.18061004e-02,  8.37593060e-03, -2.97900033e-03,  1.88477244e-02,
       -1.19465571e-02,  8.01936816e-03,  2.23685112e-02,  2.41048839e-02,
       -1.34804647e-03,  3.99642065e-02,  7.62677100e-03,  2.41736900e-02,
       -2.48665493e-02, -1.66609825e-03,  1.55024733e-02, -8.01916886e-03,
        2.47087050e-02, -2.20886301e-02,  2.80129258e-02, -5.22466656e-03,
       -3.06047089e-02,  1.80803072e-02,  1.40398005e-02, -1.33491792e-02,
       -1.63455047e-02, -1.15277879e-02,  8.97113886e-03, -9.03999340e-03,
        2.28786021e-02, -2.35637440e-03,  2.79036500e-02,  2.29201298e-02,
       -2.19189189e-02, -8.25787801e-03,  2.11426467e-02,  4.08543274e-02,
        3.79125308e-03,  7.13006244e-04, -6.61121635e-03,  6.18836749e-03,
       -7.73954438e-04, -

In [45]:
dusk_till_dawn = movie_to_idx["From Dusk Till Dawn (1996)"]
dusk_till_dawn_vector = als_model.item_factors[dusk_till_dawn]
dusk_till_dawn_vector

array([-1.12658292e-02, -2.84365332e-03, -2.98787700e-03,  2.00058185e-02,
        2.89800353e-02,  4.01085801e-03,  1.25455158e-02, -2.24686470e-02,
       -1.60430353e-02, -6.28326554e-03,  2.94677857e-02,  9.86311398e-03,
        1.55198071e-02,  2.06756126e-02,  2.09162962e-02,  9.59440321e-03,
        7.40755908e-03, -1.83186140e-02,  1.21939844e-02,  1.15832379e-02,
        5.01753949e-03,  7.13981735e-03,  1.30805559e-03,  2.94336248e-02,
        3.31746857e-03,  3.82430181e-02,  8.28397460e-03,  3.04914825e-02,
        2.99152415e-02,  1.14646507e-02, -2.88501778e-03, -1.27945486e-02,
        1.22107603e-02, -4.22668410e-03, -7.25865318e-03,  5.48434258e-03,
        1.86610911e-02, -9.94420145e-03, -6.71215868e-03,  6.37407822e-04,
        5.84975630e-03,  2.31185760e-02,  9.24287923e-03,  1.11538768e-02,
       -1.10897087e-02, -4.69422527e-03,  1.59660764e-02,  1.29574612e-02,
        5.74490335e-03, -7.39995157e-03,  1.26220463e-02, -4.67847148e-03,
        1.22902147e-03,  

- 싫어하는 영화와의 예측 선호도

In [46]:
np.dot(user_vector, star_wars_epi_4_vector)

-0.006428417

- 좋아하는 영화와의 예측 선호도


In [47]:
np.dot(user_vector, dusk_till_dawn_vector)

0.3634861

좋아한다고 넣은 영환데 0.34밖에 나오지 않음...

## 좋아하는 영화와 비슷한 영화 추천받기

In [48]:
favorite_movie = "From Dusk Till Dawn (1996)"
movie_title = movie_to_idx[favorite_movie]
similar_movies = als_model.similar_items(movie_title, N=15)
similar_movies


(array([1034,  174, 1697, 1008, 1748, 2617,  999, 2610, 1615, 1794, 1666,
        1087, 1602, 1303, 1779], dtype=int32),
 array([0.99999994, 0.40837342, 0.39990118, 0.39337528, 0.39330828,
        0.3874942 , 0.38550118, 0.3844889 , 0.3829664 , 0.3772606 ,
        0.37411347, 0.372526  , 0.37151265, 0.37040624, 0.36901498],
       dtype=float32))

In [49]:
similar_movies[0]

array([1034,  174, 1697, 1008, 1748, 2617,  999, 2610, 1615, 1794, 1666,
       1087, 1602, 1303, 1779], dtype=int32)

In [50]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
similar = [idx_to_movie[i] for i in similar_movies[0]]

좋아하는 영화와 비슷한 영화들의 title과 유사도

In [51]:
score = [i for i in similar_movies[1]]
pd.DataFrame({'movie': similar, 'similar_score' : score})

,movie,similar_score
0,From Dusk Till Dawn (1996),1.000000
1,Blade (1998),0.408373
2,Vampires (1998),0.399901
3,Tales from the Hood (1995),0.393375
4,"Prophecy, The (1995)",0.393308
5,Tales from the Crypt Presents: Bordello of Blo...,0.387494
6,Evil Dead II (Dead By Dawn) (1987),0.385501
7,Candyman (1992),0.384489
8,Halloween: The Curse of Michael Myers (1995),0.382966
9,Hellbound: Hellraiser II (1988),0.377261


유사한 영화들과 그에 대한 유사도를 dataframe으로 출력하는 함수 생성

In [52]:
def get_similar_movie(movie_name: str):
    movie_title = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_title, N=15)
    similar_movie = [idx_to_movie[i] for i in similar_movies[0]]
    similar_score = [i for i in similar_movies[1]]
    df = pd.DataFrame({'movie' : similar_movie, 'similar_score' : similar_score})
    return df

In [53]:
get_similar_movie('Deep Impact (1998)')

,movie,similar_score
0,From Dusk Till Dawn (1996),1.000000
1,Blade (1998),0.408373
2,Vampires (1998),0.399901
3,Tales from the Hood (1995),0.393375
4,"Prophecy, The (1995)",0.393308
5,Tales from the Crypt Presents: Bordello of Blo...,0.387494
6,Evil Dead II (Dead By Dawn) (1987),0.385501
7,Candyman (1992),0.384489
8,Halloween: The Curse of Michael Myers (1995),0.382966
9,Hellbound: Hellraiser II (1988),0.377261


## 내가 좋아할만한 영화 추천받기

In [54]:
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data[user], N=20, filter_already_liked_items=True)
movie_recommended

(array([1090,  831,  855,  440, 1107,  997,  456, 1368,  174,  919,  299,
         136, 1709,  839, 1486,  316,   72, 1433, 1078,  707], dtype=int32),
 array([0.26042938, 0.25216436, 0.23136052, 0.21639954, 0.20032561,
        0.19193426, 0.1882608 , 0.1881172 , 0.17100796, 0.16434093,
        0.16412285, 0.14864099, 0.14137608, 0.13760872, 0.13652024,
        0.13517219, 0.13495237, 0.13401255, 0.13301285, 0.13284689],
       dtype=float32))

추천 영화와 점수 확인

In [55]:
recommended = [idx_to_movie[i] for i in movie_recommended[0]]
score = [i for i in movie_recommended[1]]
pd.DataFrame({'movie': recommended, 'score' : score})

,movie,score
0,Nikita (La Femme Nikita) (1990),0.260429
1,"Devil's Advocate, The (1997)",0.252164
2,"Grifters, The (1990)",0.231361
3,True Romance (1993),0.216400
4,Rosemary's Baby (1968),0.200326
5,Scream (1996),0.191934
6,Arachnophobia (1990),0.188261
7,Henry: Portrait of a Serial Killer (1990),0.188117
8,Blade (1998),0.171008
9,Mulholland Falls (1996),0.164341


In [56]:
item_movie = movie_to_idx['Kansas City (1996)']
explain = als_model.explain(user, csr_data, itemid=item_movie)

In [57]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[("Jacob's Ladder (1990)", 0.008132915839490006),
 ('Devil in a Blue Dress (1995)', 0.007996980021819469),
 ('Body Snatchers (1993)', 0.0070245968214670395),
 ('From Dusk Till Dawn (1996)', 0.003459319406466598),
 ('Professional, The (a.k.a. Leon: The Professional) (1994)',
  -0.015398545089461497)]

## 회고

이번 EX는 진행할 때 Implicit 패키지의 버전이 LMS랑 맞지가 않아서 오류가 엄청 많이 떴다.  
진행 하면서 혈압이 너무 올랐는데 공식문서 살펴보면서 하다보니까 끝까지 진행은 다 했다.  
근데 은근 유사도나 추천 정도(?)가 너무 낮아서 왜 그런지가 조금 의아하긴 하다. 특별한 전처리를 할 것도 없어 보이는데..허허  
그리고 이번 MF모델을 이용할 때 그냥 user_id, movie_id, ratings자료만 사용했는데 영화의 genre도 한 번 사용해보고 싶어서 어떻게 쓸 방법이 없나? 하면서 이것저것 찾아봤다.  
genre를 정수로 바꿔보기도 하고 전처리를 했는데 이거를 이용할 수 있는 방법이 생각이 나질 않아서 결국 쓰지는 못했다... 분명히 찾아보면 방법이 있을 것이지만 그거는 나중에 공부하면서 알아보도록 하자고 다짐하면서 배운 방법으로만 진행을 했다.  
방학 기간이나 주말에 혼자서라도 kaggle notebook에서 감성 분석을 통한 추천이나 다른 방법들을 이용한 추천모델을 필사를 하면서 공부 해보고 다른 데이터로도 한 번 만들어 봐야겠다!

- 참고  
https://benfred.github.io/implicit/tutorial_lastfm.html